# Import software libraries and load the dataset #

In [ ]:
import sys                             # Read system parameters.
import numpy as np                     # Work with multi-dimensional arrays and matrices.
from numpy.random import seed
import pandas as pd                    # Manipulate and analyze data frames.
import sklearn                         # Perform feature engineering and machine learning.
import tensorflow                      # Train neural networks for deep learning.
import keras                           # Provide a frontend for TensorFlow.
from keras import datasets
import matplotlib                      # Create charts.
import matplotlib.pyplot as plt
import seaborn as sb                   # Streamline charting.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))
print('- TensorFlow {}'.format(tensorflow.__version__))
print('- Keras {}'.format(keras.__version__))
print('- Matplotlib {}'.format(matplotlib.__version__))
print('- Seaborn {}'.format(sb.__version__))
print('- Python {}\n'.format(sys.version))

# Load the dataset.
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
print('Loaded {} training records.'.format(len(X_train.data)))
print('Loaded {} testing records.'.format(len(X_test.data)))

# Comment the following two lines to make outcomes stochastic, or supply different seed values.
seed(10)
tensorflow.random.set_seed(10)

# Get acquainted with the dataset

In [ ]:
# Show dimensions of the training and testing sets and their labels.
print('Shape of data used for training and testing:\n')
print('Training data:   {}'.format(X_train.shape))
print('Training labels: {}\n'.format(y_train.shape))
print('Testing data:    {}'.format(X_test.shape))
print('Testing labels:  {}'.format(y_test.shape))

# Visualize the data examples

In [ ]:
# Show a preview of the first 20 images in a subplot grid.
# Include the class label for each one.
fig, axes = plt.subplots(nrows = 4, ncols = 5, figsize = (10, 10))

for i, ax in zip(range(25), axes.flatten()):
    ax.imshow(X_train[i,:,:], cmap = 'gray')  # Use a grayscale color map.
    ax.title.set_text('Class: {}'.format(y_train[i]))
    
fig.tight_layout()

# Prepare the data for training with Keras

In [ ]:
# Reshape arrays to add grayscale flag.
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

from keras.utils import to_categorical

# One-hot encode the data for each label.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Print the one-hot encoding for the first image.
print('One-hot encoding for first image: {}'.format(y_train[0]))

# Split the datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Split the training and validation datasets and their labels.
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  random_state = 50)

# Print shape of training and validation sets.
print(f'Training features:         {X_train.shape}')
print(f'Validation features:       {X_val.shape}')
print(f'Training labels:           {y_train.shape}')
print(f'Validation labels:         {y_val.shape}')

# Build the CNN structure

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense

# Create the model.
cnn = Sequential()

# Add model layers as specified.
cnn.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape = (28, 28, 1)))
cnn.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
cnn.add(Flatten())
cnn.add(Dense(10, activation = 'softmax'))

# Compile the model and summarize the layers

In [ ]:
# Compile the model.
cnn.compile(optimizer = 'adam',
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])

# Summarize the layers.
cnn.summary()

# Plot a graph of the model

In [ ]:
# Plot a graph of the model.
from keras.utils import plot_model
plot_model(cnn, show_shapes = True)

# Train the model

In [ ]:
# Train the model over 1 epoch.
cnn_trained = cnn.fit(X_train, y_train,
                      validation_data = (X_val, y_val),
                      epochs = 1,
                      verbose = 1)

# Evaluate the model on the test data

In [ ]:
# Evaluate the model on the test data, showing loss and accuracy.
eval_test = cnn.evaluate(X_test, y_test, verbose = 0)

print('Loss: {}'.format(round(eval_test[0], 2)))
print('Accuracy: {:.0f}%'.format(eval_test[1] * 100))

# Make predictions on the test data

In [ ]:
# Make predictions on the test data.
prediction = cnn.predict(X_test)
prediction = np.argmax(np.round(prediction), axis = 1)  # Extract class number from one-hot-encoded array.
actual = np.argmax(np.round(y_test), axis = 1)

# Print the first 20 example predictions.
print('First 20 example predictions:')
print('Actual class:    {}'.format(actual[:30]))
print('Predicted class: {}'.format(prediction[:30]))

# Visualize the predictions for 20 examples

In [ ]:
# Using the test set, show the first 20 predictions, highlighting any incorrect predictions in color.
fig, axes = plt.subplots(nrows = 4, ncols = 5, figsize = (10, 10))

for i, ax in zip(range(20), axes.flatten()):
    if actual[i] == prediction[i]:
        ax.imshow(X_test[i].reshape(28, 28), cmap = 'gray')
    else:
        ax.imshow(X_test[i].reshape(28, 28))
        
    ax.title.set_text('Actual: {}\nPredicted: {}'.format(actual[i], prediction[i]))
    
fig.tight_layout()